In [18]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import subprocess

In [19]:
train_results_dir = os.getcwd() + "/../002-Training-models/train_results/"
!ls {train_results_dir}

model_yolov5_tf_no_imgsz_800_epochs_100_batch_16_dataset_v1_loss_SGD_lr_01
model_yolov5_tf_no_imgsz_800_epochs_50_batch_16_dataset_v1_loss_SGD_lr_01
model_yolov5_tf_yolov5s_imgsz_640_epochs_100_batch_16_dataset_v1_loss_SGD_lr_001
model_yolov5_tf_yolov5s_imgsz_640_epochs_100_batch_16_dataset_v1_loss_SGD_lr_01
model_yolov5_tf_yolov5s_imgsz_640_epochs_50_batch_16_dataset_v1_loss_SGD_lr_01
model_yolov5_tf_yolov5s_imgsz_800_epochs_100_batch_16_dataset_v1_loss_SGD_lr_01
model_yolov5_tf_yolov5s_imgsz_800_epochs_50_batch_16_dataset_v1_loss_SGD_lr_01
model_yolov7_tf_yolov7training_imgsz_640_epochs_100_batch_16_dataset_v1_loss_SGD_lr_001
model_yolov7_tf_yolov7training_imgsz_640_epochs_100_batch_16_dataset_v1_loss_SGD_lr_01
model_yolov7_tf_yolov7training_imgsz_640_epochs_50_batch_16_dataset_v1_loss_SGD_lr_001
model_yolov7_tf_yolov7training_imgsz_800_epochs_100_batch_8_dataset_v1_loss_SGD_lr_01
model_yolov8_tf_no_imgsz_800_epochs_100_batch_16_dataset_v1_loss_SGD_lr_01
model_yolov8_tf_no_imgsz_800_

In [20]:
eval_results_dir = os.getcwd()
dataset_dir = os.getcwd()+"/../datasets/"
output_file = "logs/eval_vs_randomized_clips.txt"

In [21]:
dataset_dir

'/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models/../datasets/'

In [22]:
# Download YOLOv8
!pip install ultralytics

from IPython import display
display.clear_output()

%cd {eval_results_dir}
!mkdir yolov8

from ultralytics import YOLO
print('YOLOv8 setup complete.')

/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models
mkdir: yolov8: File exists
YOLOv8 setup complete.


# Evaluate vs randomized videos

In [23]:
for folder_name in sorted(os.listdir(train_results_dir)):
    if not folder_name.startswith("model"):
        continue

    folder_path = os.path.join(train_results_dir, folder_name, "weights")

    # Split the folder name into variables
    variables = folder_name.split('_')

    # Extract the required variables
    model = variables[1]
    tf = variables[3]
    imgsz = variables[5]
    epochs = variables[7]
    batch = variables[9]
    dataset = variables[11]
    loss = variables[13]
    lr = variables[15]
    
    print(folder_name)
    print("Model:", model)
    print("TF:", tf)
    print("Imgsz:", imgsz)
    print("Epochs:", epochs)
    print("Batch:", batch)
    print("Dataset:", dataset)
    print("Executing command...")
    print()
    
    if dataset == "v2":
        dataset_location = dataset_dir + model + "_guns_only_no_empty"
        
    command = None
    
    if model == "yolov5" and dataset == "v2":
        %cd yolov5
        best_pt_path = os.path.join(folder_path, "best.pt")
        command = f"python3 val.py --weights {best_pt_path} --data {dataset_location}/data.yaml --task test --name {folder_name}_rc"
        print(command)

    elif model == "yolov7" and dataset == "v2":
        %cd yolov7
        best_pt_path = os.path.join(folder_path, "best.pt")
        command = f"python3 test.py --weights {best_pt_path} --data {dataset_location}/data.yaml --task test --name {folder_name}_rc"
        print(command)

    elif model == "yolov8" and dataset == "v2":
        %cd yolov8
        best_pt_path = os.path.join(folder_path, "best.pt")
        command = f"yolo val model={best_pt_path} data={dataset_location}/data.yaml split=test name={folder_name}_rc"
        print(command)

    if command:
        # Run the command using subprocess.Popen and capture the output
        with subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True) as process:
            # Open the output file for writing
            with open(f"{eval_results_dir}/{output_file}", "a") as file:
                file.write(folder_name)
                file.write(f"\nModel: {model}")
                file.write(f"\nTF: {tf}")
                file.write(f"\nImgsz: {imgsz}")
                file.write(f"\nEpochs: {epochs}")
                file.write(f"\nBatch: {batch}")
                file.write(f"\nDataset: {dataset}")
                file.write(f"\nLoss: {loss}")
                file.write(f"\nLr0: {lr}\n")
                # Read and display each line of the output as it becomes available
                for line in process.stdout:
                    print(line, end="")
                    file.write(line)
        %cd ..

    from IPython import display
    display.clear_output()
    

print("Executing command...", "DONE!")
print()

Executing command... DONE!



In [24]:
eval_results_dir = os.getcwd()
dataset_dir = os.getcwd()+"/../datasets/"

def process_file(file_path):
    results = list()
    curr_res = dict()

    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        if line.startswith("Model:"):
            curr_res["model"] = line.split()[1]
        elif line.startswith("TF:"):
            curr_res["tf"] = line.split()[1]
        elif line.startswith("Imgsz:"):
            curr_res["imgsz"] = line.split()[1]
        elif line.startswith("Epochs:"):
            curr_res["epochs"] = line.split()[1]
        elif line.startswith("Batch:"):
            curr_res["batch"] = line.split()[1]
        elif line.startswith("Dataset:"):
            curr_res["dataset"] = line.split()[1]
        elif line.startswith("Loss:"):
            curr_res["loss_function"] = line.split()[1]
        elif line.startswith("Lr0:"):
            try:
                curr_res["lr0"] = float("0."+line.split()[1])
            except:
                curr_res["lr0"] = float(line.split()[1])
            
        if line.strip(" ").startswith("all"):
            P, R, mAP5, mAP95 = line.split()[3:]
            curr_res["all_P"] = P
            curr_res["all_R"] = R
            curr_res["all_F1"] = (float(P) * float(R)) / (float(P) + float(R))
            curr_res["all_mAP@.5"] = mAP5
            curr_res["all_mAP@.5:.95"] = mAP95

        elif line.strip(" ").startswith("Results saved to"):
            model_key = line.split("/")[-1].replace('\x1b[0m', '').replace('\n', '')
            model_info = model_key.split("_")

            curr_res["model_key"] = model_key

            results.append(curr_res)
            curr_res = dict()

    df = pd.DataFrame(results)
    df.set_index('model_key', inplace=True)
    return df

file_path = os.path.join(eval_results_dir, output_file)
df = process_file(file_path)
df.to_csv(f'{eval_results_dir}/output_csvs/results_val_vs_randomized_clips.csv')
df

,model,tf,imgsz,epochs,batch,dataset,loss_function,lr0,all_P,all_R,all_F1,all_mAP@.5,all_mAP@.5:.95
model_key,,,,,,,,,,,,,
model_yolov8_tf_yolov8m_imgsz_640_epochs_100_batch_16_dataset_v2_loss_SGD_lr_01_rc,yolov8,yolov8m,640,100,16,v2,SGD,0.01,0.576,0.335,0.211811,0.366,0.145
model_yolov8_tf_yolov8m_imgsz_640_epochs_50_batch_16_dataset_v2_loss_SGD_lr_01_rc,yolov8,yolov8m,640,50,16,v2,SGD,0.01,0.331,0.345,0.168928,0.285,0.114
model_yolov8_tf_yolov8s_imgsz_640_epochs_300_batch_16_dataset_v2_loss_SGD_lr_01_rc,yolov8,yolov8s,640,300,16,v2,SGD,0.01,0.509,0.311,0.193048,0.325,0.137
model_yolov8_tf_yolov8s_imgsz_800_epochs_100_batch_16_dataset_v2_loss_SGD_lr_01_rc,yolov8,yolov8s,800,100,16,v2,SGD,0.01,0.578,0.302,0.198359,0.314,0.124
